In [1]:
import codecs
import re
import nltk
import collections
import numpy as np
import subprocess
import json
import multiprocessing
import os
import gc
import sklearn
import networkx as nx
import scipy
import metric_learn
import itertools
import pandas as pd
import pandas
from IPython.display import display, HTML
from ipywidgets import FloatProgress

In [2]:
import sqlite3
sql_conn = sqlite3.connect('books.db')
c = sql_conn.cursor()

In [5]:
def try_drop_table(name):
    try:
        c.execute("drop table %s;" % name)
    except:
        print("no table %s" % name)

In [12]:
try_drop_table("authors")
try_drop_table("books")
try_drop_table("words")

no table authors
no table books
no table words


In [13]:
try_drop_table("chunks_50000")
try_drop_table("chunks_100000")

no table chunks_50000
no table chunks_100000


In [3]:
def char_range(start, end):
    return [chr(char) for char in range(ord(start), ord(end) + 1)]
mystem_pos = ['A', 'ADV', 'ADVPRO', 'ANUM', 'APRO', 'COM', 'CONJ', 'INTJ', 'NUM', 'PART', 'PR', 'S', 'SPRO', 'V', 'Vприч', 'Vдеепр']
alpha_chars = char_range('а', 'я') + char_range('А', 'Я') + ['ё', 'Ё']
num_chars = char_range('0', '9')
punct_chars = list(',.?!-:;"\'')
all_chars = alpha_chars + num_chars + punct_chars
most_freq_words = ['и', 'в', 'не', 'он', 'на', 'я', 'что', 'тот', 'быть', 'с', 'а', 'весь', 'это', 'как', 'она', 'по', 'но', 'они', 'к', 'у', 'ты', 'из', 'мы', 'за', 'вы', 'так', 'же', 'от', 'сказать', 'этот', 'который', 'мочь', 'человек', 'о', 'один', 'еще', 'бы', 'такой', 'только', 'себя', 'свое', 'какой', 'когда', 'уже', 'для', 'вот', 'кто', 'да', 'говорить', 'год', 'знать', 'мой', 'до', 'или', 'если', 'время', 'рука', 'нет', 'самый', 'ни', 'стать', 'большой', 'даже', 'другой', 'наш', 'свой', 'ну', 'под', 'где', 'дело', 'есть', 'сам', 'раз', 'чтобы', 'два', 'там', 'чем', 'глаз', 'жизнь', 'первый', 'день', 'тута', 'во', 'ничто', 'потом', 'очень', 'со', 'хотеть', 'ли', 'при', 'голова', 'надо', 'без', 'видеть', 'идти', 'теперь', 'тоже', 'стоять', 'друг', 'дом']
len(['avg_sent_len', 'avg_word_len', 'rel_voc_size', 'hapax_legomena_freq'] +
                  ['pos_freq_%s'%p for p in mystem_pos] + 
                  ['char_freq_%s'%c for c in all_chars] + 
                  ['word_freq_%s'%w for w in most_freq_words] +
                  ['bipos_freq_%s_%s'%(p1, p2) for p1 in mystem_pos for p2 in mystem_pos])

461

In [4]:
c.execute("""
    create table authors (
        id integer primary key,
        name text
    );""")
c.execute("""
    
    create table books (
        id integer primary key,
        author integer,
        name text
    );
""")
c.execute("""
    
    create table words (
        id integer primary key,
        sent integer,
        book integer,
        text text,
        lex text,
        wt double,
        pos text,
        gr text
    );
""")
c.execute("""
    create index if not exists index_sent ON words (sent);
""")
c.execute("""
    create index if not exists index_book ON words (book);
""")
sql_conn.commit()

In [5]:
def progressify(iterable):
    l = list(iterable)
    progress = FloatProgress(min=0, max=len(l))
    display(progress)
    for it in l:
        progress.value += 1
        yield it

In [16]:
def author(book):
    return book.split('_')[0]
            
def get_json_from_file(file):
    with codecs.open(file, encoding='utf-8') as f:
        content = f.read()
        return [json.loads(l) for l in content.splitlines()]
    
def get_rows(word_analysis):
    sentence = 1
    for i, w in enumerate(word_analysis):
        #print(sentence)
        if w['text'] == '\\s':
            sentence += 1
        else:
            analysis = w.get('analysis')
            if analysis:
                a = analysis[0]
                lex = a['lex']
                wt = a['wt']
                gr = a['gr']
                pos = re.split('[,=()]', gr)[0]
                yield (sentence, w['text'], lex, wt, pos, gr)
            else:
                yield (sentence, w['text'], None, None, None, None)

for file in progressify(os.listdir("json")):
    book_author = author(file)
    author_id_row = c.execute("select id from authors where name = ?", (book_author,)).fetchone()
    author_id = author_id_row[0] if author_id_row else c.execute(
        "insert into authors (name) values (?)", (book_author,)).lastrowid
    book_id = c.execute("insert into books (name, author) values (?, ?)", (file, author_id)).lastrowid
    word_analysis = get_json_from_file(os.path.join("json", file))
    rows = list(get_rows(word_analysis))
    c.executemany("""insert into words (sent, text, lex, wt, pos, gr, book)
                  values (?, ?, ?, ?, ?, ?, %s)""" % book_id, rows)
sql_conn.commit()

In [6]:
def get_chunks(book_id, chunk_size):
    words = list(c.execute("select id, text, lex, pos, gr, wt, sent from words where book = ?", (book_id,)))
    df = pandas.DataFrame(words, columns='id, text, lex, pos, gr, wt, sent'.split(', '))
    df['len'] = df.text.str.len()
    df = df[(df.len < 50) & (df.sent > 1) & (df.sent < df.iloc[-1].sent)]
    #df['cum_len'] = df.len.cumsum()
    sent_len = df.groupby('sent').len.sum().to_frame().len.cumsum().to_frame()
    sent_len['chunk'] = sent_len.len // chunk_size
    df['chunk'] = df.sent.map(sent_len.chunk)
    df = df.set_index(['chunk', 'sent'])
    chunks = df.index.get_level_values('chunk').unique()
    return [df.loc[c,] for c in chunks]

def write_chunk_boundaries(chunk_size):
    table = 'chunks_%s' % chunk_size
    print(table)
    try_drop_table(table)
    c.execute("""
    
    create table %s (
        id integer primary key,
        begin integer,
        end integer
    );
    """ % table)
    for book_id, in progressify(c.execute("select id from books")):
        chunks = get_chunks(book_id, chunk_size)
        c.executemany("""insert into %s (begin, end)
                  values (?, ?)""" % table, [(int(c.id.iloc[0]), int(c.id.iloc[-1])) for c in chunks])
    sql_conn.commit()

write_chunk_boundaries(50000)

chunks_50000
no table chunks_50000


In [9]:
write_chunk_boundaries(100000)

chunks_100000


In [10]:
sql_conn.commit()

In [5]:
list(c.execute("select * from words where book = '231' limit 1000"))

[(35775951, 1, 231, 'child', None, None, None, None),
 (35775952, 1, 231, '_', None, None, None, None),
 (35775953, 1, 231, 'prose', None, None, None, None),
 (35775954, 1, 231, ' ', None, None, None, None),
 (35775955, 1, 231, 'Григорий', 'григорий', 0.0, 'S', 'S,имя,муж,од=им,ед'),
 (35775956, 1, 231, ' ', None, None, None, None),
 (35775957,
  1,
  231,
  'Бенционович',
  'бенционович',
  0.0,
  'S',
  'S,отч,муж,од=им,ед'),
 (35775958, 1, 231, ' ', None, None, None, None),
 (35775959, 1, 231, 'Остер', 'острый', 87.2, 'A', 'A=ед,кр,муж'),
 (35775960, 1, 231, ' ', None, None, None, None),
 (35775961, 1, 231, 'Сказка', 'сказка', 53.9, 'S', 'S,жен,неод=им,ед'),
 (35775962, 1, 231, ' ', None, None, None, None),
 (35775963, 1, 231, 'с', 'с', 11151.7, 'PR', 'PR='),
 (35775964, 1, 231, ' ', None, None, None, None),
 (35775965,
  1,
  231,
  'подробностями',
  'подробность',
  44.7,
  'S',
  'S,жен,неод=твор,мн'),
 (35775966, 1, 231, '\n', None, None, None, None),
 (35775967, 1, 231, '\n', 

In [7]:
sql_conn.close()